In [1]:
import pulp
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary, LpStatus
from pulp.apis import PULP_CBC_CMD
import pandas as pd

# Параметры
NUM_BUSES = 8
DAYS_OF_WEEK = range(7)  # Дни недели (0: Понедельник, ..., 6: Воскресенье)
DRIVERS_COUNT = 8
NUM_8HR_DRIVERS = 4  # Половина водителей работает по 8 часов
NUM_12HR_DRIVERS = DRIVERS_COUNT - NUM_8HR_DRIVERS
TIME_SLOTS = range(24)  # Часы суток

# Инициализация задачи
model = LpProblem(name="bus-scheduling", sense=LpMinimize)

# Переменные
x = LpVariable.dicts(
    "x", [(d, b, day, t) for d in range(DRIVERS_COUNT)
          for b in range(NUM_BUSES) for day in DAYS_OF_WEEK for t in TIME_SLOTS],
    cat=LpBinary
)

# Ограничения

# 1. Каждый автобус должен иметь одного водителя на каждый час дня
for b in range(NUM_BUSES):
    for day in DAYS_OF_WEEK:
        for t in TIME_SLOTS:
            model += lpSum(x[d, b, day, t] for d in range(DRIVERS_COUNT)) == 1, f"Bus_{b}_Day_{day}_Hour_{t}"

# 2. Водитель должен оставаться на одном автобусе в течение своей смены
for d in range(DRIVERS_COUNT):
    shift_duration = 8 if d < NUM_8HR_DRIVERS else 12
    for day in DAYS_OF_WEEK:
        for start in range(24 - shift_duration + 1):
            for b in range(NUM_BUSES):
                model += lpSum(x[d, b, day, t] for t in range(start, start + shift_duration)) in [0, shift_duration], \
                    f"Driver_{d}_Day_{day}_Bus_{b}_Shift_{start}"

# 3. Ограничения для 8-часовых водителей (работают с понедельника по пятницу)
for d in range(NUM_8HR_DRIVERS):
    for day in DAYS_OF_WEEK:
        if day >= 5:  # Выходные
            model += lpSum(x[d, b, day, t] for b in range(NUM_BUSES) for t in TIME_SLOTS) == 0, f"8hr_Rest_{d}_Day_{day}"

# 4. Ограничения для 12-часовых водителей (работают 1 день, отдыхают 2 дня)
for d in range(NUM_12HR_DRIVERS, DRIVERS_COUNT):
    start_day = (d - NUM_12HR_DRIVERS) % 3
    for day in DAYS_OF_WEEK:
        if (day - start_day) % 3 == 0:
            model += lpSum(x[d, b, day, t] for b in range(NUM_BUSES) for t in TIME_SLOTS) == 12, \
                f"12hr_Work_{d}_Day_{day}"
        else:
            model += lpSum(x[d, b, day, t] for b in range(NUM_BUSES) for t in TIME_SLOTS) == 0, \
                f"12hr_Rest_{d}_Day_{day}"

# 5. Один водитель не может работать на нескольких автобусах одновременно
for d in range(DRIVERS_COUNT):
    for day in DAYS_OF_WEEK:
        for t in TIME_SLOTS:
            model += lpSum(x[d, b, day, t] for b in range(NUM_BUSES)) <= 1, f"Driver_{d}_Day_{day}_Hour_{t}_SingleBus"

# Целевая функция: минимизация количества активных водителей
model += lpSum(x[d, b, day, t] for d in range(DRIVERS_COUNT)
               for b in range(NUM_BUSES) for day in DAYS_OF_WEEK for t in TIME_SLOTS), "MinimizeDrivers"

# Решение задачи
status = model.solve(PULP_CBC_CMD(msg=1))

# Отображение результата
print(f"Solver status: {LpStatus[status]}")

# Формирование расписания
schedule = pd.DataFrame(index=[f"Driver {d + 1}" for d in range(DRIVERS_COUNT)],
                        columns=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

for d in range(DRIVERS_COUNT):
    for day in DAYS_OF_WEEK:
        assigned_buses = [b for b in range(NUM_BUSES) if any(x[d, b, day, t].varValue for t in TIME_SLOTS)]
        schedule.iloc[d, day] = f"Buses: {', '.join(map(str, assigned_buses))}" if assigned_buses else "Rest"

print(schedule)

Solver status: Infeasible
                     Monday               Tuesday             Wednesday  \
Driver 1        Buses: 5, 7        Buses: 0, 3, 4        Buses: 2, 4, 7   
Driver 2     Buses: 2, 5, 6  Buses: 1, 2, 3, 4, 5     Buses: 2, 3, 4, 5   
Driver 3     Buses: 4, 5, 6     Buses: 1, 2, 3, 5  Buses: 1, 2, 5, 6, 7   
Driver 4        Buses: 1, 3     Buses: 1, 2, 6, 7     Buses: 1, 3, 4, 5   
Driver 5  Buses: 1, 2, 3, 4                  Rest                  Rest   
Driver 6               Rest           Buses: 6, 7                  Rest   
Driver 7               Rest                  Rest           Buses: 1, 2   
Driver 8     Buses: 2, 3, 4                  Rest                  Rest   

                   Thursday             Friday        Saturday  \
Driver 1        Buses: 5, 7  Buses: 3, 4, 6, 7            Rest   
Driver 2        Buses: 5, 6  Buses: 1, 2, 3, 4            Rest   
Driver 3     Buses: 1, 4, 6  Buses: 1, 2, 3, 4            Rest   
Driver 4     Buses: 1, 3, 5       